<a href="https://colab.research.google.com/github/luisosmx/web-scraperf1/blob/main/poc_etl_f1_teams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime


# GET request to URL:
url = "https://www.formula1.com/en/results.html/2023/team.html"
response = requests.get(url)

In [2]:
def data_extraction(url: str) -> pd.DataFrame:

  response = requests.get(url)

  # Analizar el contenido HTML usando BeautifulSoup
  soup = BeautifulSoup(response.content, "html.parser")
  # Encontrar la tabla de resultados para los conductores
  table = soup.find("table", attrs={"class": "resultsarchive-table"})

  # Extraer los datos de la tabla en un DataFrame de pandas
  df = pd.read_html(str(table))[0]
  return df

In [3]:
def data_tranformation(df: pd.DataFrame) -> pd.DataFrame:

  # Seleccionar sólo las columnas necesarias
  df = df[["Pos", "Team", "PTS"]]

  # Agregar columan de "Date"
  now = datetime.now()

  epoch_time = int(now.timestamp())
    
  df["Date"] = epoch_time

  new_name = {"Pos": "Position", "PTS": "Points"}
  df = df.rename(columns=new_name)

  # Convertir la columna "PTS" a un tipo de datos de cadena
  df["Points"] = df["Points"].astype(str)
  df["Date"] = df["Date"].astype(str)
  df["Position"] = df["Position"].astype(str)

  return df

In [4]:
def load_data(df: pd.DataFrame) -> str:
  client = bigquery.Client()

  # TODO(developer): Set table_id to the ID of the table to create.
  table_id = "formula1-data-382219.data_F1.results_drivers"


  job_config = bigquery.LoadJobConfig(
      # Specify a (partial) schema. All columns are always written to the
      # table. The schema is used to assist in data type definitions.
      schema=[
          # Specify the type of columns whose type cannot be auto-detected. For
          # example the "title" column uses pandas dtype "object", so its
          # data type is ambiguous.
          bigquery.SchemaField("Position", bigquery.enums.SqlTypeNames.STRING)
          
      ],
      # Optionally, set the write disposition. BigQuery appends loaded rows
      # to an existing table by default, but with WRITE_TRUNCATE write
      # disposition it replaces the table with the loaded data.
      write_disposition="WRITE_TRUNCATE",
  )

  job = client.load_table_from_dataframe(
      df, table_id, job_config=job_config
  )  # Make an API request.
  job.result()  # Wait for the job to complete.

  table = client.get_table(table_id)  # Make an API request.
  print(
      "Loaded {} rows and {} columns to {}".format(
          table.num_rows, len(table.schema), table_id
      )
  )

In [5]:
def main():
  raw_data = data_extraction(url)
  transformed_data = data_tranformation(raw_data)
  result = load_data(transformed_data)
  print(result)
  return result

In [ ]:
main()